# QUANTUM PHASE ESTIMATION

## IMPORTS and SETUP

In [32]:
# general imports
import numpy as np
import math
import matplotlib.pyplot as plt
# magic word for producing visualizations in notebook
%matplotlib inline

# AWS imports: Import Amazon Braket SDK modules
from braket.circuits import Circuit, circuit
from braket.devices import LocalSimulator
from braket.aws import AwsDevice

# local imports
from utils_qpe import *
from utils import Plotter, DeviceUtils,BraketTaskScanner, DeviceScanner
from qiskit.quantum_info import TwoQubitBasisDecomposer

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
# set up device: local simulator or the on-demand simulator
device = LocalSimulator()
#device = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/sv1")

### Pauli Matrices:
In some of our examples, we choose the unitary $U$ to be given by the **Pauli Matrices**, which we thus define as follows:

In [34]:
# Define Pauli matrices
Id = np.eye(2)             # Identity matrix
X = np.array([[0., 1.],
              [1., 0.]])   # Pauli X
Y = np.array([[0., -1.j],
              [1.j, 0.]])  # Pauli Y
Z = np.array([[1., 0.],
              [0., -1.]])  # Pauli Z

In [35]:
# set total number of qubits
precision_qubits = range(3)
query_qubits = [3]

# prepare query register
my_qpe_circ = Circuit().h(query_qubits)

# set unitary
unitary = X

# show small QPE example circuit
my_qpe_circ = my_qpe_circ.qpe(precision_qubits, query_qubits, unitary)
#print('QPE CIRCUIT:')
#print(my_qpe_circ)

In [37]:
out = run_qpe(
    unitary,
    precision_qubits,
    query_qubits,
    my_qpe_circ,
    device,
    items_to_keep=1,
    shots=1000)

In [38]:
postprocess_qpe_results(out)

Measurement counts: Counter({'0000': 503, '0001': 497})
Results in precision register: {'000': 1000}
QPE phase estimates: [0.0]
QPE eigenvalue estimates: [1.+0.j]


# TEST 

In this example, we choose the unitary to be a _random_ two-qubit unitary, diagonal in the computational basis. We initialize the query register to be in the eigenstate $|11\rangle$ of $U$, which we can prepare using that $|11\rangle = X\otimes X|00\rangle$.
In this case we should be able to read off the eigenvalue and phase from $U$ and verify that QPE gives the right answer.

In [39]:
# Generate a random 2 qubit unitary matrix:
from scipy.stats import unitary_group

# Fix random seed for reproducibility
np.random.seed(seed=46)

# Get random two-qubit unitary
random_unitary = unitary_group.rvs(2**2)

# Let's diagonalize this
evals = np.linalg.eig(random_unitary)[0]

# Since we want to be able to read off the eigenvalues of the unitary in question
# let's choose our unitary to be diagonal in this basis
unitary = np.diag(evals)

# Check that this is indeed unitary, and print it out:
print('Two-qubit random unitary:\n', np.round(unitary, 3))
print('Check for unitarity: ', np.allclose(np.eye(len(unitary)), unitary.dot(unitary.T.conj())))

# Print eigenvalues
print('Eigenvalues:', np.round(evals, 3))

Two-qubit random unitary:
 [[ 0.865-0.502j  0.   +0.j     0.   +0.j     0.   +0.j   ]
 [ 0.   +0.j    -0.345-0.939j  0.   +0.j     0.   +0.j   ]
 [ 0.   +0.j     0.   +0.j    -0.001+1.j     0.   +0.j   ]
 [ 0.   +0.j     0.   +0.j     0.   +0.j    -0.836+0.549j]]
Check for unitarity:  True
Eigenvalues: [ 0.865-0.502j -0.345-0.939j -0.001+1.j    -0.836+0.549j]


Autovalore che ci aspettiamo di ricevere, considerando una precisione data $0.001$ con un errore $\epsilon = 5\% $:

In [40]:
numero_dcimali_stima = 3
max_error = 0.05
n_binary_estimation = -int(np.floor(np.log2(10**-numero_dcimali_stima)))


In [41]:
target_eigenvalue = evals[-1]
print('Target eigenvalue:', np.round(target_eigenvalue, numero_dcimali_stima))
target_phase = np.angle(target_eigenvalue,deg=False)/(2*np.pi)
print('Target phase:', np.round(target_phase, numero_dcimali_stima))

Target eigenvalue: (-0.836+0.549j)
Target phase: 0.408


In [42]:
# Set total number of precision qubits
number_precision_qubits = get_number_precision_qubits(n_binary_estimation,max_error)

# Define the set of precision qubits
precision_qubits = range(number_precision_qubits)

# Define the query qubits. We'll have them start after the precision qubits
query_qubits = [number_precision_qubits, number_precision_qubits+1]

# State preparation for eigenstate |1,1> of diagonal U
query = Circuit().x(query_qubits[0]).x(query_qubits[1])

# Run the test with U=X
out = run_qpe(unitary, precision_qubits, query_qubits, query, device)

postprocess_qpe_results(out)
# Postprocess results

eigenvalues = out['eigenvalues']
print('QPE eigenvalue estimates:', np.round(eigenvalues, 5))

# compare output to exact target values
print('Target eigenvalue:', np.round(evals[-1], 5))

Measurement counts: Counter({'0110100001011011': 64, '0110100001010111': 22, '0110100000101011': 2, '0110100001011111': 2, '0110100001001011': 1, '0110100001010011': 1, '0110100001001111': 1, '0110100000110011': 1, '0110100001000011': 1, '0110100001100111': 1, '0110100011100111': 1, '0110011101111011': 1, '0110011111110011': 1, '0110100100100011': 1})
Results in precision register: {'01101000001100': 1, '01100111111100': 1, '01101000010110': 64, '01101000001010': 2, '01100111011110': 1, '01101000111001': 1, '01101000011001': 1, '01101000010100': 1, '01101001001000': 1, '01101000010010': 1, '01101000010011': 1, '01101000010101': 22, '01101000010111': 2, '01101000010000': 1}
QPE phase estimates: [0.4075927734375]
QPE eigenvalue estimates: [-0.83613+0.54854j]
QPE eigenvalue estimates: [-0.83613+0.54854j]
Target eigenvalue: (-0.83606+0.54864j)


# Let s try with Rigetti


In [15]:
device = DeviceUtils.get_device("rigetti")

In [19]:
DeviceScanner(device=device).get_supported_gates()

['cz',
 'xy',
 'ccnot',
 'cnot',
 'cphaseshift',
 'cphaseshift00',
 'cphaseshift01',
 'cphaseshift10',
 'cswap',
 'h',
 'i',
 'iswap',
 'phaseshift',
 'pswap',
 'rx',
 'ry',
 'rz',
 's',
 'si',
 'swap',
 't',
 'ti',
 'x',
 'y',
 'z',
 'start_verbatim_box',
 'end_verbatim_box']

## L'esempio su QPU è quanto di più semplice, scegliamo una fase casuale e la mettiamo su un qubit mediante una rotazione di fase, proviamo poi a stimarla mediante la QPU
### N.b. $R(\phi) = e^{2 \pi i  \phi}$

In [53]:
np.random.seed(seed=58)

phi = np.random.rand()
print(phi)


0.3651055830226939


In [54]:
@circuit.subroutine(register=True)
def cphase_shift_rnd(control_qubit,target_qubit):
    return Circuit().cphaseshift(control=control_qubit, target=target_qubit,angle=2*np.pi*phi)


In [55]:
numero_dcimali_stima = 2
max_error = 0.05
n_binary_estimation = -int(np.floor(np.log2(10**-numero_dcimali_stima)))

In [56]:
device=LocalSimulator()

In [57]:
unitary = cphase_shift_rnd(control_qubit=0,target_qubit=1).as_unitary()[2:,2:]

target_eigenvalue = unitary[-1,-1]
print('Target eigenvalue:', np.round(target_eigenvalue, numero_dcimali_stima))
target_phase = np.angle(target_eigenvalue,deg=False)/(2*np.pi)
print('Target phase:', np.round(target_phase, numero_dcimali_stima))


Target eigenvalue: (-0.66+0.75j)
Target phase: 0.37


In [58]:
# Set total number of precision qubits
number_precision_qubits = get_number_precision_qubits(n_binary_estimation,max_error)

# Define the set of precision qubits
precision_qubits = range(number_precision_qubits)

# Define the query qubits. We'll have them start after the precision qubits
query_qubits = [number_precision_qubits]

# State preparation for eigenstate |1,1> of diagonal U
query = Circuit().x(query_qubits[0])
print(number_precision_qubits)

11


In [159]:
# Run the test with U=X
out = run_qpe(cphase_shift_rnd, precision_qubits, query_qubits, query, device)

postprocess_qpe_results(out)
# Postprocess results

eigenvalues = out['eigenvalues']
print('QPE eigenvalue estimates:', np.round(eigenvalues, 5))

# compare output to exact target values
print('Target eigenvalue:', target_eigenvalue)

Measurement counts: Counter({'010111011001': 793, '010111010111': 93, '010111011011': 40, '010111010101': 17, '010111011101': 8, '010111010011': 5, '010111011111': 4, '010111100001': 3, '010111010001': 3, '010111000011': 2, '010111001101': 2, '010110111011': 2, '010111100011': 2, '010111001001': 2, '010111101101': 2, '010111001111': 2, '010111101011': 2, '010111110011': 1, '010111110111': 1, '100011111111': 1, '010111111101': 1, '010110000001': 1, '010111100101': 1, '010111101001': 1, '010110101111': 1, '011000011101': 1, '010111001011': 1, '010111000111': 1, '011000000101': 1, '010111100111': 1, '010111110001': 1, '010100100001': 1, '010000100001': 1, '011000000011': 1, '010111101111': 1})
Results in precision register: {'01011100100': 2, '01011111001': 1, '01100000001': 1, '01011100101': 1, '01011110100': 1, '01100000010': 1, '01011000000': 1, '01011101101': 40, '01011110000': 3, '01011110010': 1, '01011101011': 93, '01011100011': 1, '01011100001': 2, '01011100110': 2, '01011101110':

# RIGETTI PER DAVVERO


In [18]:
device = DeviceUtils.get_device("rigetti")

In [ ]:
# Run the test with U=X
out = run_qpe(cphase_shift_rnd, precision_qubits, query_qubits, query, device)

postprocess_qpe_results(out)
# Postprocess results

eigenvalues = out['eigenvalues']
print('QPE eigenvalue estimates:', np.round(eigenvalues, 5))

# compare output to exact target values
print('Target eigenvalue:', target_eigenvalue)

In [59]:
task_id = 'arn:aws:braket:us-west-1:058017682654:quantum-task/a2d3eeb1-da4e-4b5c-8022-7178341d5f7f'

In [60]:
scanner_aspen = BraketTaskScanner(task_id)


In [63]:
task=scanner_aspen.get_task()
out = get_and_analyze_result(task,12,precision_qubits,items_to_keep=4)

In [64]:
postprocess_qpe_results(out)

# compare output to exact target values
print('Target eigenvalue:', target_eigenvalue, '\n target phase: ',phi)

Measurement counts: Counter({'110011010101': 15, '110011001101': 12, '110011000101': 11, '100011000101': 11, '110011011100': 10, '010001010100': 10, '100011010100': 10, '100011011100': 10, '000011000100': 10, '000011011100': 9, '100010000101': 9, '010011011101': 9, '110001011100': 8, '110011000100': 8, '000011001100': 8, '010011010101': 8, '100011011101': 8, '100011000100': 8, '110011011101': 7, '110010010100': 7, '000011001101': 7, '110010000100': 7, '010011111101': 7, '110010000101': 7, '010001011101': 6, '100010001101': 6, '110001000101': 6, '010011001101': 6, '010011001100': 6, '010011010100': 6, '100010011100': 6, '100011111101': 6, '010010001101': 6, '010001111101': 5, '110010010101': 5, '000001001100': 5, '000011000101': 5, '010011000101': 5, '100011001101': 5, '010001011100': 5, '110010001100': 5, '000001010100': 5, '010010000100': 5, '010011011100': 5, '100011010101': 5, '110000011101': 5, '000011011101': 5, '010001001100': 5, '110011001100': 5, '100011111100': 4, '11000110010